# Lesson 1
## Backtesting simple strategy (easy way)

## Description

1. Asset: 
  - XAUUSD

2. Period: 
  - 2024/06/01 - 2025/01/01

3. Entry conditions:
  - Time Frame: 30min
  - buy at 10:30 AM UTC, if prev candle closed bullish.
  - sell at 12:00 AM UTC, if prev candle closed bearish.

4. Exit conditions:
  - exit after 1 hour.

5. Sizing: 
  - 1lot.

## 0. Imports

In [28]:
import pandas as pd
import vectorbt as vbt

## 1. Data preparation

In [41]:
data = pd.read_csv("data.csv", parse_dates=["Time"])
data

,Time,Open,High,Low,Close,Volume
0,2024-06-02 21:00:00+00:00,2325.76,2326.95,2325.76,2326.95,2
1,2024-06-02 22:00:00+00:00,2328.77,2329.12,2322.70,2323.37,64
2,2024-06-02 22:30:00+00:00,2323.38,2326.56,2323.38,2326.56,68
3,2024-06-02 23:00:00+00:00,2325.30,2326.58,2324.20,2325.57,68
4,2024-06-02 23:30:00+00:00,2325.39,2326.04,2323.90,2325.55,67
...,...,...,...,...,...,...
6984,2024-12-31 17:30:00+00:00,2624.20,2625.24,2623.07,2624.14,80
6985,2024-12-31 18:00:00+00:00,2623.21,2624.92,2623.15,2624.19,19
6986,2024-12-31 22:00:00+00:00,2624.19,2624.19,2624.19,2624.19,1
6987,2025-01-01 23:00:00+00:00,2624.80,2625.01,2623.18,2623.57,78


## 2. Entries / Exits 

In [42]:
long_entries = pd.Series(False, index=data.index)
short_entries = pd.Series(False, index=data.index)
long_exits = pd.Series(False, index=data.index)
short_exits = pd.Series(False, index=data.index)

In [96]:
is_bullish_opened = False
is_bearish_opened = False

for i, candle_data in enumerate(data.itertuples()):
  dt = candle_data.Time
  time_ = str(dt.time()).split(":")

  if is_bullish_opened == True:
    if float(time_[0]) == 11 and float(time_[1]) == 30:
      long_exits.iloc[i] = True
      is_bullish_opened = False

  if is_bearish_opened == True:
    if float(time_[0]) == 13 and float(time_[1]) == 0:
      short_exits.iloc[i] = True
      is_bearish_opened = False

  if float(time_[0]) == 10 and float(time_[1]) == 30: # for bullish entry
    prev_row = data.iloc[i-1]
    prev_row_open = float(prev_row["Open"].item())
    prev_row_close = float(prev_row["Close"].item())

    if prev_row_open < prev_row_close: # bullish
      long_entries.iloc[i] = True
      is_bullish_opened = True

  if float(time_[0]) == 12 and float(time_[1]) == 0: # for bearish entry
    prev_row = data.iloc[i-1]
    prev_row_open = float(prev_row["Open"].item())
    prev_row_close = float(prev_row["Close"].item())

    if prev_row_open > prev_row_close: # bearish
      short_entries.iloc[i] = True
      is_bearish_opened = True
      

In [97]:
data[long_exits == True]

,Time,Open,High,Low,Close,Volume
75,2024-06-04 11:30:00+00:00,2325.95,2327.96,2325.77,2327.85,77
263,2024-06-10 11:30:00+00:00,2298.72,2304.98,2298.72,2304.90,75
310,2024-06-11 11:30:00+00:00,2306.96,2307.52,2305.69,2307.38,79
404,2024-06-13 11:30:00+00:00,2311.02,2311.32,2304.74,2307.07,78
451,2024-06-14 11:30:00+00:00,2329.10,2336.20,2328.81,2333.01,78
...,...,...,...,...,...,...
6696,2024-12-20 11:30:00+00:00,2601.40,2604.64,2601.09,2604.01,75
6790,2024-12-24 11:30:00+00:00,2613.27,2615.84,2612.39,2614.92,78
6831,2024-12-26 11:30:00+00:00,2629.45,2629.45,2627.00,2627.77,80
6878,2024-12-27 11:30:00+00:00,2625.68,2628.71,2625.68,2628.65,77


## 3. Backtest

In [98]:

pf = vbt.Portfolio.from_signals(
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    price=data["Close"],
    entries=long_entries,
    exits=long_exits,
    short_entries=short_entries,
    short_exits=short_exits,
    size=1.0,
    fees=0.001,
    init_cash=10000,
    freq="1min"
)

print(pf.stats())
pf.plot()


Start                                       0
End                                      6988
Period                        4 days 20:29:00
Start Value                           10000.0
End Value                          9173.62842
Total Return [%]                    -8.263716
Benchmark Return [%]                12.751886
Max Gross Exposure [%]              29.759743
Total Fees Paid                     764.67158
Max Drawdown [%]                     8.293253
Max Drawdown Duration         4 days 20:00:00
Total Trades                              151
Total Closed Trades                       151
Total Open Trades                           0
Open Trade PnL                            0.0
Win Rate [%]                        16.556291
Best Trade [%]                        0.72456
Worst Trade [%]                     -1.361597
Avg Winning Trade [%]                0.103222
Avg Losing Trade [%]                 -0.28206
Avg Winning Trade Duration    0 days 00:02:00
Avg Losing Trade Duration     0 da

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '851eb969-3919-461a-968c-91d4af3764af',
              'x': array([   0,    1,    2, ..., 6986, 6987, 6988], shape=(6989,)),
              'xaxis': 'x',
              'y': array([2326.95, 2323.37, 2326.56, ..., 2624.19, 2623.57, 2623.68],
                         shape=(6989,)),
              'yaxis': 'y'},
             {'customdata': array([[  1.     ,   1.     ,   2.33358],
                                   [  2.     ,   1.     ,   2.32898],
                                   [  5.     ,   1.     ,   2.34224],
                                   ...,
                                   [297.     ,   1.     ,   2.62767],
                                   [298.     ,   1.     ,   2.62661],
                                   [300.     ,   1.     ,   2.61067]], shape=(151, 3)),
  

In [100]:
data.iloc[31]

Time      2024-06-03 13:00:00+00:00
Open                        2334.56
High                        2335.79
Low                          2330.3
Close                       2333.58
Volume                           79
Name: 31, dtype: object